## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [39]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Cherskiy,RU,2020-09-27 00:36:55,68.75,161.30,42.35,69,0,19.66,clear sky
1,1,Ostrovnoy,RU,2020-09-27 00:36:56,68.05,39.51,42.39,87,100,14.20,overcast clouds
2,2,College,US,2020-09-27 00:36:56,64.86,-147.80,53.60,58,40,3.36,scattered clouds
3,3,Farafangana,MG,2020-09-27 00:36:56,-22.82,47.83,67.51,86,9,9.71,clear sky
4,4,Estelle,US,2020-09-27 00:36:56,29.85,-90.11,79.00,94,1,3.36,clear sky


In [40]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip (in F)? "))
max_temp = float(input("What is the maximum temperature you would like for your trip (in F)? "))


What is the minimum temperature you would like for your trip (in F)? 80
What is the maximum temperature you would like for your trip (in F)? 100


In [41]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
18,18,Hilo,US,2020-09-27 00:27:07,19.73,-155.09,84.20,62,90,5.82,overcast clouds
21,21,Saint George,US,2020-09-27 00:35:17,37.10,-113.58,91.40,8,1,14.99,clear sky
32,32,Doha,QA,2020-09-27 00:36:31,25.29,51.53,87.80,70,0,6.93,clear sky
34,34,Kapaa,US,2020-09-27 00:37:02,22.08,-159.32,86.00,58,1,11.41,clear sky
35,35,Galveston,US,2020-09-27 00:32:54,29.37,-94.97,82.99,65,1,9.17,clear sky
38,38,Banda Aceh,ID,2020-09-27 00:37:03,5.56,95.32,80.76,78,99,8.95,overcast clouds
39,39,Butaritari,KI,2020-09-27 00:27:10,3.07,172.79,82.27,74,98,10.58,overcast clouds
41,41,Soure,BR,2020-09-27 00:37:04,-0.72,-48.52,80.01,79,90,12.44,overcast clouds
43,43,San Ignacio,BZ,2020-09-27 00:37:04,17.16,-89.07,82.40,88,75,4.70,broken clouds
45,45,Cabo San Lucas,MX,2020-09-27 00:37:04,22.89,-109.91,84.99,74,20,6.93,few clouds


In [42]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        127
City           127
Country        127
Date           127
Lat            127
Lng            127
Max Temp       127
Humidity       127
Cloudiness     127
Wind Speed     127
Description    127
dtype: int64

In [43]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()


City_ID        127
City           127
Country        127
Date           127
Lat            127
Lng            127
Max Temp       127
Humidity       127
Cloudiness     127
Wind Speed     127
Description    127
dtype: int64

In [44]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
18,Hilo,US,84.20,overcast clouds,19.73,-155.09,
21,Saint George,US,91.40,clear sky,37.10,-113.58,
32,Doha,QA,87.80,clear sky,25.29,51.53,
34,Kapaa,US,86.00,clear sky,22.08,-159.32,
35,Galveston,US,82.99,clear sky,29.37,-94.97,
38,Banda Aceh,ID,80.76,overcast clouds,5.56,95.32,
39,Butaritari,KI,82.27,overcast clouds,3.07,172.79,
41,Soure,BR,80.01,overcast clouds,-0.72,-48.52,
43,San Ignacio,BZ,82.40,broken clouds,17.16,-89.07,
45,Cabo San Lucas,MX,84.99,few clouds,22.89,-109.91,


In [45]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]


    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [46]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
18,Hilo,US,84.20,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
21,Saint George,US,91.40,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
32,Doha,QA,87.80,clear sky,25.29,51.53,Doha Marriott Hotel
34,Kapaa,US,86.00,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
35,Galveston,US,82.99,clear sky,29.37,-94.97,Comfort Inn & Suites Texas City
...,...,...,...,...,...,...,...
667,Daru,PG,82.24,scattered clouds,-9.08,143.21,New century hotel
669,Pangkalanbuun,ID,81.43,overcast clouds,-2.68,111.62,Grand Kecubung Hotel
674,Ixtapa,MX,86.00,overcast clouds,20.70,-105.20,Container Inn Puerto Vallarta
675,Noumea,NC,82.00,overcast clouds,-22.28,166.46,Hôtel Le Lagon


In [47]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotels_df = hotel_df.dropna()
clean_hotels_df.head(5)


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
18,Hilo,US,84.20,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
21,Saint George,US,91.40,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
32,Doha,QA,87.80,clear sky,25.29,51.53,Doha Marriott Hotel
34,Kapaa,US,86.00,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
35,Galveston,US,82.99,clear sky,29.37,-94.97,Comfort Inn & Suites Texas City


In [48]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotels_df.to_csv("WeatherPy_Vacation.csv", index_label="City_ID")

In [56]:

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
</dl>
"""



# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotels_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotels_df[["Lat", "Lng"]]



In [54]:
# 11a. Add a marker layer for each city to the map. 



# 11b. Display the figure

fig2 = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig2.add_layer(marker_layer)

# Call the figure to plot the data.
fig2

Figure(layout=FigureLayout(height='420px'))